## Setup


[Setup Python Virtual Environment in VSCode](/venv_setup.md)


**Security Note:** Using fixed package versions ensures stability and reproducibility, but may expose you to security vulnerabilities if not regularly updated. Balance stability and security by periodically reviewing and updating packages.

In [ ]:
# Install required packages in a virtual environment (Currently only tested with Python 3.12.4)
%pip install --upgrade pip
%pip install pandas==2.2.2
%pip install matplotlib==3.9.2
%pip install requests==2.32.3

# Download JLCPCB Parts CSV File 

In [1]:
import requests
import os


def download_file(url, filename):
    try:
        # Check if the file already exists
        if os.path.exists(filename):
            # Delete the existing file
            os.remove(filename)
            print(f"Deleted existing file: {filename}")

        response = requests.get(f"{url}/{filename}", stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filename, "wb") as f:
            for chunk in response.iter_content(None):
                f.write(chunk)
        print(f"Downloaded {url}/{filename} to {filename}")
    except requests.RequestException as e:
        print(f"Download {url} failed: {e}")


# URL
url = "https://cdfer.github.io/jlcpcb-parts-database"

# Local file path to save the database file
local_file = "jlcpcb-components-basic-preferred.csv"

# Download the file
download_file(url, local_file)

Deleted existing file: jlcpcb-components-basic-preferred.csv
Downloaded https://cdfer.github.io/jlcpcb-parts-database/jlcpcb-components-basic-preferred.csv to jlcpcb-components-basic-preferred.csv


In [6]:
missing_footprints = {}

import json
import re
import pandas as pd

def extract_value(description, mode, lcsc_id):
    if mode == "Resistors":
        pattern = r"(\d+(?:[a-zA-Z]*)?(?:Ω|ohm|Ohm|OHM))"  # matches numbers followed by Ω, ohm, Ohm, or OHM
    elif mode == "Capacitors":
        pattern = r"(\d+(?:[a-zA-Z]*)?(?:F|f|Farad|farad|pF|pf|nF|nf|uF|uf))"  # matches numbers followed by F, f, Farad, farad, pF, pf, nF, nf, uF, uf
    else:
        raise ValueError("extract_value(): Invalid mode")

    match = re.search(pattern, description, re.IGNORECASE)
    if match:
        return match.group(0)
    else:
        print(f"Error: No value found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
        return None

def extract_diode_type(description, pins, lcsc_id):
    diode_types = {
        "Schottky": {"pins": 2, "type": "Schottky"},
        "Recovery": {"pins": 2, "type": "Recovery"},
        "General": {"pins": 2, "type": "General"},
        "Rectifiers": {"pins": 4, "type": "Bridge-Rectifier"},
        "Switching": {"pins": 2, "type": "Switching"},
        "Zener": {"pins": 2, "type": "Zener"},
        "Bidirectional": {"pins": 2, "type": "TVS-Bi"},
        "Unidirectional": {"pins": 2, "type": "TVS-Uni"}
    }
    for keyword, diode_info in diode_types.items():
        if keyword.casefold() in description.casefold():
            if diode_info["pins"] == pins:
                return diode_info["type"]
            else:
                print(f"Error: Number of pins ({pins}) does not match expected ({diode_info['pins']}) for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
                return None
    print(f"Error: No diode type found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
    return None

def extract_transistor_type(description, pins, lcsc_id):
    transistor_types = {
        "PNP": {"pins": 3, "type": "PNP"},
        "NPN": {"pins": 3, "type": "NPN"},
        "NChannel": {"pins": 3, "type": "NMOS"},
        "PChannel": {"pins": 3, "type": "PMOS"},
        "N-Channel": {"pins": 3, "type": "NMOS"},
        "P-Channel": {"pins": 3, "type": "PMOS"}
    }
    for keyword, transistor_info in transistor_types.items():
        if keyword.casefold() in description.casefold():
            if transistor_info["pins"] == pins:
                return transistor_info["type"]
            else:
                print(f"Error: Number of pins ({pins}) does not match expected ({transistor_info['pins']}) for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
                return None
    print(f"Error: No transistor type found for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
    return None

def extract_LED_value(description, lcsc_id):
    if lcsc_id == 2985996:
        return "Red", "LED"
    elif lcsc_id == 34499:
        return "White", "LED"
    
    color_pattern = r"(Red|Green|Blue|Yellow|White|Emerald)"
    bi_color_pattern = r"(Red,Blue|Red,Emerald)"

    color_match = re.search(color_pattern, description, re.IGNORECASE)
    bi_color_match = re.search(bi_color_pattern, description, re.IGNORECASE)

    if bi_color_match:
        color = bi_color_match.group(0).replace("Red,Emerald", "Red/Green").replace("Red,Blue", "Red/Blue")
        return color, "LED-Bi-Colour"
    elif color_match:
        color = color_match.group(0).replace("Emerald", "Green")
        return color, "LED"
    else:
        print(f"Error: No value extracted for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
        return None, None

def get_plan_type(df, index):
    if df.loc[index, "basic"] > 0:
        return "Basic Component"
    elif df.loc[index, "preferred"] > 0:
        return "Preferred Component"
    else:
        print("extended component found")
        return "Extended Component"


def generate_header(name):
    symbol = f'\t(symbol "{name}"'
    symbol += "\n\t\t(pin_numbers hide)"
    symbol += "\n\t\t(pin_names\n\t\t\t(offset 0)\n\t\t)"
    symbol += "\n\t\t(exclude_from_sim no)"
    symbol += "\n\t\t(in_bom yes)"
    symbol += "\n\t\t(on_board yes)"
    return symbol


def generate_property(key, value, at, size=1.27, hide="yes", autoplace=True):
    if autoplace == False:
        autoplace_str = "(do_not_autoplace)"
    else:
        autoplace_str = ""
    return f'\n\t\t(property "{key}" "{value}"\n\t\t\t(at {at}){autoplace_str}\n\t\t\t(effects\n\t\t\t\t(font\n\t\t\t\t\t(size {size} {size})\n\t\t\t\t)\n\t\t\t\t(hide {hide})\n\t\t\t)\n\t\t)'


def generate_rectangle(
    start,
    end,
    name,
    index,
    stroke={"width": 0.254, "type": "default"},
    fill={"type": "none"},
):
    return f'\n\t\t(symbol "{name}_{index}_1"\n\t\t\t(rectangle\n\t\t\t\t(start {start})\n\t\t\t\t(end {end})\n\t\t\t\t(stroke\n\t\t\t\t\t(width {stroke["width"]})\n\t\t\t\t\t(type {stroke["type"]})\n\t\t\t\t)\n\t\t\t\t(fill\n\t\t\t\t\t(type {fill["type"]})\n\t\t\t\t)\n\t\t\t)\n\t\t)'


def generate_polyline(
    points,
    name,
    index,
    stroke={"width": 0.254, "type": "default"},
    fill={"type": "none"},
):
    polyline_str = f'\n\t\t(symbol "{name}_{index}_1"\n\t\t\t(polyline\n\t\t\t\t(pts\n'
    for point in points:
        polyline_str += f"\t\t\t\t\t(xy {point})\n"
    polyline_str += f'\t\t\t\t)\n\t\t\t\t(stroke\n\t\t\t\t\t(width {stroke["width"]})\n\t\t\t\t\t(type {stroke["type"]})\n\t\t\t\t)\n\t\t\t\t(fill\n\t\t\t\t\t(type {fill["type"]})\n\t\t\t\t)\n\t\t\t)\n\t\t)'
    return polyline_str


def generate_pin_pair(pin_type, name, index, pin_length = "1.27", pinA=1, pinB=2):
    symbol = f'\n\t\t(symbol "{name}_{index}_1"\n\t\t\t(pin {pin_type}\n\t\t\t\t(at 0 3.81 270)\n\t\t\t\t(length {pin_length})\n\t\t\t\t(name "~"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t\t(number "{pinA}"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t)'
    symbol += f'\n\t\t\t(pin {pin_type}\n\t\t\t\t(at 0 -3.81 90)\n\t\t\t\t(length {pin_length})\n\t\t\t\t(name "~"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t\t(number "{pinB}"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t)'
    symbol += "\n\t\t)"
    return symbol


def generate_kicad_symbol(
    mode,
    secondary_mode,
    lcsc,
    datasheet,
    description,
    footprint,
    value,
    keywords,
    price,
    component_class,
    stock,
    category,
    manufacturer,
    manufacturerPartID,
    attributes,
    units,
):
    if mode == "Resistors":
        ref_designator = "R"
        ref_position = "2.032 0 0"
        value_position = "0 0 90"
        value_autoplace = False
        name = f"{footprint},{value}"
    elif mode == "Capacitors":
        ref_designator = "C"
        ref_position = "2.032 0 0"
        value_position = "3.556 -1.524 0"
        value_autoplace = True
        name = f"{footprint},{value}"
    elif mode == "Diodes":
        ref_designator = "D"
        ref_position = "2.032 0.834 0"
        value_position = "2.032 -1.2122 0"
        value_autoplace = True
        if secondary_mode == "LED" or secondary_mode == "LED-Bi-Colour":
            name = f"{secondary_mode},{footprint},{value}"
        else:
            name = f"{value},{manufacturerPartID}"
            value = manufacturerPartID
    elif mode == "Transistors":
        ref_designator = "Q"
        ref_position = "4.8514 0.834 0"
        value_position = "4.8514 -1.2122 0"
        value_autoplace = True
        # Remove brackets from manufacturerPartID
        cleaned_manufacturerPartID = manufacturerPartID.replace('(', '').replace(')', '').replace('RANGE:',' ')
        name = f"{value},{cleaned_manufacturerPartID}"
        value = cleaned_manufacturerPartID
            
    else:
        ref_designator = "NA"
        ref_position = "0 0 0"
        value_position = "0 0 0"
        value_autoplace = True
        name = f"{footprint},{value}"
    lcsc = f"C{lcsc}" 
    if f"{ref_designator}_{footprint}" in footprints_lookup:
        footprint = f"JLCPCB-Kicad-Footprints:{ref_designator}_{footprint}"
    else:
        missing_footprint = f"{ref_designator}_{footprint}"
        if missing_footprint in missing_footprints:
            missing_footprints[missing_footprint] += 1
        else:
            missing_footprints[missing_footprint] = 1

    symbol = generate_header(name)

    symbol += generate_property("Reference", ref_designator, ref_position, hide="no")
    symbol += generate_property("Value", value, value_position, size=0.8, hide="no", autoplace=value_autoplace)
    symbol += generate_property("Footprint", footprint, "-1.778 0 90")
    symbol += generate_property("Datasheet", datasheet, "0 0 0")
    symbol += generate_property("Description", description, "0 0 0")
    symbol += generate_property("LCSC", lcsc, "0 0 0")
    symbol += generate_property("Price", price, "0 0 0")
    symbol += generate_property("Class", component_class, "0 0 0")
    symbol += generate_property("Stock", stock, "0 0 0")
    symbol += generate_property("Category", category, "0 0 0")
    symbol += generate_property("Manufacturer", manufacturer, "0 0 0")
    symbol += generate_property("Manufacturer Part ID", manufacturerPartID, "0 0 0")

    if type(attributes) == dict:
        for key, value in attributes.items():
            symbol += generate_property(f"{key}", f"{value}", "0 0 0")

    symbol += generate_property("ki_keywords", keywords, at="0 0 0")
    symbol += generate_property("ki_fp_filters", f"{ref_designator}_*", "0 0 0")

    if mode == "Resistors":
        symbol += generate_rectangle("-1.016 2.54", "1.016 -2.54", name=name, index=0)
        for i in range(1, units + 1):
            symbol += generate_pin_pair("passive line", name, i, "1.27", i, (units * 2) - (i - 1))

    elif mode == "Capacitors":
        symbol += generate_polyline(["-1.27 0.635", "1.27 0.635"], name=name, index=0)
        symbol += generate_polyline(["-1.27 -0.635", "1.27 -0.635"], name=name, index=0)
        polarized_footprints = [
            "C_Plugin,D5xL11mm",
            "C_Plugin,D10xL12.5mm",
            "C_CASE-B-3528-21(mm)",
            "C_CASE-A-3216-18(mm)"
        ]
        if any(s in footprint for s in polarized_footprints):
            symbol += generate_polyline(["-1.27 1.27", "-0.635 1.27"], name=name, index=0, stroke={ "width": 0.127,"type": "default" })
            symbol += generate_polyline(["-0.9525 1.5875", "-0.9525 0.9525"], name=name, index=0, stroke={ "width": 0.127,"type": "default" })
        for i in range(1, units + 1):
            symbol += generate_pin_pair("passive line", name, i, "3.175", i, (units * 2) - (i - 1))

    elif mode == "Diodes":
        if secondary_mode == "Bridge-Rectifier":
            symbol += generate_rectangle("-2.54 2.54", "2.54 -2.54", name=name, index=0)
            
        elif secondary_mode == "TVS-Bi":
            symbol += generate_polyline(["-1.27 2.54", "0 0", "1.27 2.54", "-1.27 2.54"], name=name, index=0)
            symbol += generate_polyline(["1.27 -2.54", "0 0", "-1.27 -2.54", "1.27 -2.54"], name=name, index=0)
            symbol += generate_polyline(["-1.905 -0.635", "-1.27 0", "1.27 0", "1.905 0.635"], name=name, index=0)
            for i in range(1, units + 1):
                symbol += generate_pin_pair("passive line", name, i, "3.81", (units * 2) - (i - 1), i)
        elif secondary_mode == "LED-Bi-Colour":
            symbol += generate_polyline(["-3.175 -1.27", "-0.635 -1.27"], name=name, index=0)
            symbol += generate_polyline(["3.175 1.27", "0.635 1.27"], name=name, index=0)
            symbol += generate_polyline(["-3.175 1.27", "-1.905 -1.27", "-0.635 1.27", "-3.175 1.27"], name=name, index=0)
            symbol += generate_polyline(["-1.905 -1.27", "-1.905 -2.54", "1.905 -2.54", "1.905 -1.27"], name=name, index=0)
            symbol += generate_polyline(["-1.905 1.27", "-1.905 2.54", "1.905 2.54", "1.905 1.27"], name=name, index=0)
            symbol += generate_polyline(["3.175 -1.27", "1.905 1.27", "0.635 -1.27", "3.175 -1.27"], name=name, index=0)
            symbol += generate_polyline(["-3.81 -1.27", "-5.334 0.254", "-5.334 -0.254", "-5.334 0.254", "-4.826 0.254"], name=name, index=0, stroke={"width": 0.127, "type": "default"})
            symbol += generate_polyline(["-3.81 0", "-5.334 1.524", "-5.334 1.016", "-5.334 1.524", "-4.826 1.524"], name=name, index=0, stroke={"width": 0.127, "type": "default"})
            symbol += generate_polyline(["3.81 0", "5.334 -1.524", "5.334 -1.016", "5.334 -1.524", "4.826 -1.524"], name=name, index=0, stroke={"width": 0.127, "type": "default"})
            symbol += generate_polyline(["3.81 1.27", "5.334 -0.254", "5.334 0.254", "5.334 -0.254", "4.826 -0.254"], name=name, index=0, stroke={"width": 0.127, "type": "default"})
            symbol += generate_pin_pair("passive line", name, 1, "1.27", 1,  2)
        else:
            symbol += generate_polyline(["-1.27 1.27", "0.00 -1.27", "1.27 1.27", "-1.27 1.27"], name=name, index=0)
            if secondary_mode == "Schottky":
                symbol += generate_polyline(["0.635 -1.905", "1.27 -1.905", "1.27 -1.27", "-1.27 -1.27", "-1.27 -0.635", "-0.635 -0.635"], name=name, index=0)
            elif secondary_mode == "Zener":
                symbol += generate_polyline(["-1.27 -1.27", "1.27 -1.27", "1.27 -0.762"], name=name, index=0)
            elif secondary_mode == "TVS-Uni":
                symbol += generate_polyline(["-1.905 -1.905", "-1.27 -1.27", "1.27 -1.27", "1.905 -0.635"], name=name, index=0)
            elif secondary_mode == "LED":
                symbol += generate_polyline(["-1.27 -1.27", "1.27 -1.27"], name=name, index=0)
                symbol += generate_polyline(["-1.905 -1.27", "-3.429 0.254", "-3.429 -0.254", "-3.429 0.254", "-2.921 0.254"], name=name, index=0, stroke={ "width": 0.127,"type": "default" })
                symbol += generate_polyline(["-1.905 0", "-3.429 1.524", "-3.429 1.016", "-3.429 1.524", "-2.921 1.524"], name=name, index=0, stroke={ "width": 0.127,"type": "default" })            
            else:
                symbol += generate_polyline(["-1.27 -1.27", "1.27 -1.27"], name=name, index=0)
            
            for i in range(1, units + 1):
                symbol += generate_pin_pair("passive line", name, i, "3.81", (units * 2) - (i - 1), i)
                
    elif mode == "Transistors":
        if secondary_mode == "NPN":
            npn = f"""		(symbol "{name}_0_1"
			(polyline
				(pts
					(xy -2.54 0) (xy 0.635 0)
				)
				(stroke
					(width 0.1524)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.635 0.635) (xy 2.54 2.54)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.794 -1.27) (xy 2.794 -1.27)
				)
				(stroke
					(width 0.1524)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.794 -1.27) (xy 2.794 -1.27)
				)
				(stroke
					(width 0.1524)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.635 -0.635) (xy 2.54 -2.54) (xy 2.54 -2.54)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.635 1.905) (xy 0.635 -1.905) (xy 0.635 -1.905)
				)
				(stroke
					(width 0.508)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 1.27 -1.778) (xy 1.778 -1.27) (xy 2.286 -2.286) (xy 1.27 -1.778) (xy 1.27 -1.778)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
			(circle
				(center 1.27 0)
				(radius 2.8194)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
		)
		(symbol "{name}_1_1"
			(pin open_collector line
				(at 2.54 5.08 270)
				(length 2.54)
				(name "C"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "1"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin input line
				(at -5.08 0 0)
				(length 2.54)
				(name "B"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "2"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin open_emitter line
				(at 2.54 -5.08 90)
				(length 2.54)
				(name "E"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "3"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
		)"""
            symbol += npn
        elif secondary_mode == "PNP":
            pnp = f"""		(symbol "{name}_0_1"
			(polyline
				(pts
					(xy -2.54 0) (xy 0.635 0)
				)
				(stroke
					(width 0.1524)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.635 0.635) (xy 2.54 2.54)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.635 -0.635) (xy 2.54 -2.54) (xy 2.54 -2.54)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.635 1.905) (xy 0.635 -1.905) (xy 0.635 -1.905)
				)
				(stroke
					(width 0.508)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.286 -1.778) (xy 1.778 -2.286) (xy 1.27 -1.27) (xy 2.286 -1.778) (xy 2.286 -1.778)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
			(circle
				(center 1.27 0)
				(radius 2.8194)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
		)
		(symbol "{name}_1_1"
			(pin open_collector line
				(at 2.54 5.08 270)
				(length 2.54)
				(name "C"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "1"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin input line
				(at -5.08 0 0)
				(length 2.54)
				(name "B"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "2"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin open_emitter line
				(at 2.54 -5.08 90)
				(length 2.54)
				(name "E"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "3"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
		)"""
            symbol += pnp
        elif secondary_mode == "NMOS":
            nmos = f"""		(symbol "{name}_0_1"
			(polyline
				(pts
					(xy 0.254 0) (xy -2.54 0)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.254 1.905) (xy 0.254 -1.905)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 -1.27) (xy 0.762 -2.286)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 0.508) (xy 0.762 -0.508)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 2.286) (xy 0.762 1.27)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.54 2.54) (xy 2.54 1.778)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.54 -2.54) (xy 2.54 0) (xy 0.762 0)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 -1.778) (xy 3.302 -1.778) (xy 3.302 1.778) (xy 0.762 1.778)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 1.016 0) (xy 2.032 0.381) (xy 2.032 -0.381) (xy 1.016 0)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
			(polyline
				(pts
					(xy 2.794 0.508) (xy 2.921 0.381) (xy 3.683 0.381) (xy 3.81 0.254)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 3.302 0.381) (xy 2.921 -0.254) (xy 3.683 -0.254) (xy 3.302 0.381)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(circle
				(center 1.651 0)
				(radius 2.794)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(circle
				(center 2.54 -1.778)
				(radius 0.254)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
			(circle
				(center 2.54 1.778)
				(radius 0.254)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
		)
		(symbol "{name}_1_1"
			(pin passive line
				(at 2.54 5.08 270)
				(length 2.54)
				(name "D"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "1"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin input line
				(at -5.08 0 0)
				(length 2.54)
				(name "G"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "2"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin passive line
				(at 2.54 -5.08 90)
				(length 2.54)
				(name "S"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "3"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
		)"""
            symbol += nmos
        elif secondary_mode == "PMOS":
            pmos = f"""		(symbol "{name}_0_1"
			(polyline
				(pts
					(xy 0.254 0) (xy -2.54 0)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.254 1.905) (xy 0.254 -1.905)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 -1.27) (xy 0.762 -2.286)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 0.508) (xy 0.762 -0.508)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 2.286) (xy 0.762 1.27)
				)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.54 2.54) (xy 2.54 1.778)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.54 -2.54) (xy 2.54 0) (xy 0.762 0)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 0.762 1.778) (xy 3.302 1.778) (xy 3.302 -1.778) (xy 0.762 -1.778)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 2.286 0) (xy 1.27 0.381) (xy 1.27 -0.381) (xy 2.286 0)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
			(polyline
				(pts
					(xy 2.794 -0.508) (xy 2.921 -0.381) (xy 3.683 -0.381) (xy 3.81 -0.254)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(polyline
				(pts
					(xy 3.302 -0.381) (xy 2.921 0.254) (xy 3.683 0.254) (xy 3.302 -0.381)
				)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(circle
				(center 1.651 0)
				(radius 2.794)
				(stroke
					(width 0.254)
					(type default)
				)
				(fill
					(type none)
				)
			)
			(circle
				(center 2.54 -1.778)
				(radius 0.254)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
			(circle
				(center 2.54 1.778)
				(radius 0.254)
				(stroke
					(width 0)
					(type default)
				)
				(fill
					(type outline)
				)
			)
		)
		(symbol "{name}_1_1"
			(pin passive line
				(at 2.54 5.08 270)
				(length 2.54)
				(name "D"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "1"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin input line
				(at -5.08 0 0)
				(length 2.54)
				(name "G"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "2"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
			(pin passive line
				(at 2.54 -5.08 90)
				(length 2.54)
				(name "S"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
				(number "3"
					(effects
						(font
							(size 1.27 1.27)
						)
					)
				)
			)
		)"""
            symbol += pmos
        else:
            symbol += generate_rectangle("-2.54 2.54", "2.54 -2.54", name=name, index=0)
            

    symbol += "\n\t)"
    return symbol


def generate_kicad_symbol_libs(symbols):
    for lib_name, symbol_list in symbols.items():
        lib_content = "(kicad_symbol_lib\n"
        lib_content += "\t(version 20231120)\n"
        lib_content += '\t(generator "CDFER")\n'
        lib_content += '\t(generator_version "8.0")\n'
        for symbol in symbol_list:
            lib_content += symbol + "\n"
        lib_content += ")\n"

        lib_content = lib_content.replace("℃", "°C")

        with open(f"JLCPCB-Kicad-Symbols/JLCPCB-{lib_name}.kicad_sym", "w") as f:
            f.write(lib_content)

df = pd.read_csv(local_file)

footprints_dir = "JLCPCB-Kicad-Footprints"
footprints_lookup = {os.path.splitext(file)[0] for file in os.listdir(footprints_dir)}

symbols = {"Resistors": [], "Capacitors": [], "Diodes":[], "Transistors":[]}
smt_joint_cost = 0.0017
hand_solder_joint_cost = 0.0173

for index in range(0, len(df)):
    # lcsc,category_id,category,subcategory,mfr,package,joints,manufacturer,basic,preferred,description,datasheet,stock,last_on_stock,price,extra
    lcsc = df.loc[index, "lcsc"]
    category = f'{df.loc[index,"category"]},{df.loc[index,"subcategory"]}'
    manufacturer = df.loc[index,"manufacturer"]
    manufacturerPartID = df.loc[index,"mfr"]
    footprint = df.loc[index, "package"]
    description = df.loc[index, "description"]
    joints = df.loc[index, "joints"]
    units = 1
    secondary_mode = ""
    subcategory = str(df.loc[index,"subcategory"])

    if "Plugin" in footprint:
        joint_cost = hand_solder_joint_cost
    else:
        joint_cost = smt_joint_cost

    price_json = json.loads(df.loc[index, "price"])
    price = float(price_json[0]["price"] + (joints * joint_cost))
    price = round(price, 3)
    price_str = f"{price}USD"

    component_class = get_plan_type(df, index)
    stock = df.loc[index, "stock"]

    keywords = ""

    value = None

    try:
        extra_json = json.loads(df.loc[index, "extra"])
        datasheet = extra_json["datasheet"]["pdf"]
        attributes = extra_json["attributes"]
    except:
        datasheet = df.loc[index, "datasheet"]
        attributes = []

    if df.loc[index, "category"] == "Resistors":
        value = extract_value(description, "Resistors", lcsc)
        if "x4" in footprint:
            units = 4
        lib_name = "Resistors"
        
    elif df.loc[index, "category"] == "Capacitors":
        value = extract_value(description, "Capacitors", lcsc)
        lib_name = "Capacitors"
        
    elif df.loc[index, "category"] == "Diodes" or ("TVS" in subcategory):
        value= extract_diode_type(description, joints, lcsc)
        secondary_mode = value
        lib_name = "Diodes"
        
    elif subcategory == "Light Emitting Diodes (LED)":
        value, secondary_mode = extract_LED_value(description,lcsc)
        lib_name = "Diodes"
        
    elif subcategory == "MOSFETs" or (subcategory == "Bipolar Transistors - BJT") or (subcategory == "Bipolar (BJT)"):
        value= extract_transistor_type(description, joints, lcsc)
        secondary_mode = value
        lib_name = "Transistors"

    if price > 3.0:
        df.drop(index = index, inplace= True)
        # print(f"Error: ${price}USD for https://jlcpcb.com/partdetail/C{lcsc}  ({description})")
    elif value != None:
        df.drop(index = index, inplace= True)
        symbols[lib_name].append(
            generate_kicad_symbol(
                lib_name,
                secondary_mode,
                lcsc,
                datasheet,
                description,
                footprint,
                value,
                keywords,
                price_str,
                component_class,
                stock,
                category,
                manufacturer,
                manufacturerPartID,
                attributes,
                units,
            )
        )
    # else:
    #     print(f"error finding value for C{lcsc} >{description}<")

df.to_csv("jlcpcb-leftover.csv", index=False)

generate_kicad_symbol_libs(symbols)

print("\nMissing Footprints:")
for footprint, count in sorted(missing_footprints.items(), key=lambda x: x[1], reverse=True):
    print(f"{footprint} x{count}")

Error: No value found for https://jlcpcb.com/partdetail/C907741  (0201  Multilayer Ceramic Capacitors MLCC - SMD/SMT ROHS)
Error: No value found for https://jlcpcb.com/partdetail/C3013473  (1812  Multilayer Ceramic Capacitors MLCC - SMD/SMT ROHS)
Error: No value found for https://jlcpcb.com/partdetail/C3008298  (Plugin,P=10mm  Safety Capacitors ROHS)
Error: Number of pins (3) does not match expected (2) for https://jlcpcb.com/partdetail/C22395518  (280A 5880W 21V 6V Bidirectional 4.5V DFN2020-3L ESD and Surge Protection (TVS/ESD) ROHS)
Error: Number of pins (4) does not match expected (2) for https://jlcpcb.com/partdetail/C22395520  (5A 75W 15V 6V Bidirectional 5V SOT-143 ESD and Surge Protection (TVS/ESD) ROHS)
Error: No diode type found for https://jlcpcb.com/partdetail/C2925443  (43.5A 400W 9.2V 6.4V 5V SMA(DO-214AC)  ESD and Surge Protection (TVS/ESD) ROHS)
Error: No diode type found for https://jlcpcb.com/partdetail/C78395  (58.1A 600W 10.5V 6.45V 5.8V SMB(DO-214AA)  ESD and Surge

In [124]:
# Calculate cost per joint for each component
df['cost_per_joint'] = (df['price'].apply(json.loads).apply(lambda x: float(x[0]['price'])) + (df['joints'] * df['package'].apply(lambda x: hand_solder_joint_cost if 'Plugin' in x else smt_joint_cost))) / df['joints']

# Filter parts with more than 3 joints
df_filtered = df[df['joints'] > 7]

# Sort the components by cost per joint
df_sorted = df_filtered.sort_values(by='cost_per_joint')

# Print the 100 components with the lowest cost per joint
for index, row in df_sorted.head(100).iterrows():
    print(f"USD/Joint = {row['cost_per_joint']:.3f} https://jlcpcb.com/partdetail/C{row['lcsc']} Joints: {row['joints']} {row['mfr']} ({row['description']})")

USD/Joint = 0.004 https://jlcpcb.com/partdetail/C7420374 Joints: 10 H5VU25U (4A 14V 56W 6V Unidirectional 5V DFN2510-10L  ESD and Surge Protection (TVS/ESD) ROHS)
USD/Joint = 0.005 https://jlcpcb.com/partdetail/C20615800 Joints: 10 H5VU25UC (5A 75W 15V 10V Unidirectional 5V DFN2510-10L  ESD and Surge Protection (TVS/ESD) ROHS)
USD/Joint = 0.007 https://jlcpcb.com/partdetail/C5605 Joints: 14 74HC14D,653 (Schmitt Trigger 6 21ns@6V,50pF 2uA 2V~6V SOIC-14  Inverters ROHS)
USD/Joint = 0.008 https://jlcpcb.com/partdetail/C5590 Joints: 14 74HC04D,653 (6 14ns@6V,50pF 2uA 2V~6V SOIC-14  Inverters ROHS)
USD/Joint = 0.008 https://jlcpcb.com/partdetail/C5947 Joints: 16 74HC595D,118 (8 2V~6V 1 Serial to serial or parallel SOIC-16  Shift Registers ROHS)
USD/Joint = 0.008 https://jlcpcb.com/partdetail/C7512 Joints: 16 ULN2003ADR (Seven channels 50V 500mA SOIC-16  Darlington Transistor Arrays ROHS)
USD/Joint = 0.008 https://jlcpcb.com/partdetail/C71035 Joints: 14 LM324DT (20nA Four Channels 1.3MHz SOI

In [12]:
import csv

descriptions = []

with open('jlcpcb-leftover.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if "inductor" in row["description"].lower():
            descriptions.append(row["description"])
            
for description in descriptions:
    current_match = re.search(r"(\d+(?:\.\d+)?)mA|(\d+(?:\.\d+)?)A", description)
    inductance_match = re.search(r"(\d+(?:\.\d+)?)nH|(\d+(?:\.\d+)?)uH", description)
    
    if current_match:
        current = float(current_match.group(1) or current_match.group(2))
        if current_match.group(1):
            current /= 1000  # convert mA to A
    else:
        current = None
    
    if inductance_match:
        inductance = float(inductance_match.group(1) or inductance_match.group(2))
        if inductance_match.group(1):
            inductance *= 0.001  # convert nH to uH
    else:
        inductance = None
    
    if current_match and inductance_match:
        energy = 0.5 * inductance * current**2
        print(f"{round(energy, 3)}J {current}A {inductance}uH")

0.0J 0.23A 0.012uH
0.0J 0.6A 0.0015uH
0.0J 0.3A 0.0008uH
0.0J 0.85A 0.0011uH
0.0J 0.33A 0.0050999999999999995uH
0.0J 0.3A 0.0039uH
0.001J 0.3A 0.015uH
0.0J 0.3A 0.01uH
0.001J 0.015A 4.7uH
0.0J 0.003A 10.0uH
0.003J 0.3A 0.068uH
0.001J 0.05A 1.0uH
0.003J 0.05A 2.2uH
0.001J 0.015A 10.0uH
0.0J 0.005A 22.0uH
0.0J 0.002A 100.0uH
0.001J 0.025A 4.7uH
0.003J 0.025A 10.0uH
0.242J 0.22A 10.0uH
12.716J 0.34A 220.0uH
3.217J 3.7A 0.47000000000000003uH
312.5J 2.5A 100.0uH
269.528J 4.95A 22.0uH
218.489J 0.73A 820.0uH
312.5J 2.5A 100.0uH
990.0J 30.0A 2.2uH
5.39J 7.0A 0.22uH
7.581J 0.43A 82.0uH
